In [35]:
from utils.another_helper import get_only_text_bboxes
from ultralytics import YOLO

model = YOLO("runs/detect/train/weights/best.pt")
source = "test/test.jpg"

bboxes = get_only_text_bboxes(model, source)
print(bboxes[4])


image 1/1 /Users/kesh/Documents/GitHub/DarkPatterns/test/test.jpg: 320x640 3 Icons, 21 Images, 23 Texts, 39.2ms
Speed: 1.2ms preprocess, 39.2ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)
tensor([[1393.6549,  847.4094, 1523.9873,  890.3303]])


In [36]:
bb = []
for i in bboxes:
    bb.append(i.tolist()[0])

print(bb[1])

[325.9818115234375, 847.961669921875, 505.6567077636719, 891.9682006835938]


In [37]:
sorted_bb = sorted(bb, key=lambda bbox: (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]))[::-1]
print(sorted_bb)

[[857.9954833984375, 248.54238891601562, 1211.41650390625, 314.69293212890625], [860.4110107421875, 357.33099365234375, 1340.0130615234375, 379.0419006347656], [325.9818115234375, 847.961669921875, 505.6567077636719, 891.9682006835938], [585.1304931640625, 850.752685546875, 766.4241333007812, 886.2340087890625], [1393.6549072265625, 847.409423828125, 1523.9873046875, 890.330322265625], [327.17108154296875, 849.050048828125, 503.2984924316406, 878.4175415039062], [36.56367874145508, 540.585693359375, 257.2926940917969, 562.3521728515625], [583.8275756835938, 852.331298828125, 762.8098754882812, 875.492431640625], [1285.09326171875, 120.34346771240234, 1479.83349609375, 140.9470672607422], [888.488525390625, 858.554931640625, 990.5933837890625, 891.8544921875], [791.0108642578125, 121.18232727050781, 957.7666625976562, 139.15928649902344], [887.1984252929688, 848.7322998046875, 990.7578735351562, 877.0426025390625], [1527.0325927734375, 120.90187072753906, 1663.2105712890625, 138.9550170

In [38]:
import cv2

def crop_image(image, coords):
    x1, y1, x2, y2 = coords
    print((x2 - x1) * (y2 - y1))
    cropped_image = image[int(y1):int(y2), int(x1):int(x2)]
    #plt.imshow(cropped_image)
    #plt.show()
    return cropped_image

img = cv2.imread('test/test.jpg')
im_list= []
for i in sorted_bb:
    im_list.append(crop_image(img, i))

import matplotlib.pyplot as plt

# Set matplotlib to display plots inline
%matplotlib inline

# Display the imag

23378.992489445955
10412.595513090491
7906.868848504499
6432.538247443736
5593.983594536781
5172.420449277386
4804.493585953955
4145.43281686306
4012.3498050123453
3400.046914368868
2997.7621878739446
2931.799330741167
2458.4409774653614
2286.2599858324975
2083.2007139846683
2077.2622480094433
1933.8635388119146
1839.036332923919
1785.4994577616453
1577.305681295693
1377.3249604811426
1341.136989964638
1149.1021390631795


In [39]:
import pytesseract

text_list = []
for image in im_list:
    text = pytesseract.image_to_string(image)
    text_list.append(text)

print(text_list)


['Redmi Note 13 Pro 5G Series\nFrom =23,.999*\n', '1.5k Curved AMOLED | Flaaship IP68 Protection\n', 'Pendrives & SD Cards\nFrom 7229\n', 'Premium PowerBanks\n\na ay\n', 'Best of Shavers\nfrom 71 G49\n', 'Pendrives & SD Cards\n', 'Best of Electronics\n', 'Premium PowerBanks\n', 'Beauty, Toys & More v\n', '20 MEMetS\nErom 27179\n', 'Home & Furniture Y\n', 'JSB Gadgets\n', 'Two Wheelers v\n', 'Electronics Vv\n', 'Projectors\n', 'Erom 29996\n', 'Printers\n', 'From 73999\n', 'Appliances\n', 'Fashion v\n', 'Grocery\n', 'Printers.\n', 'Mobiles\n']


In [40]:
import languagemodels as lm 

text_list = [p for p in text_list if any(char.isdigit() for char in p)]

price = []
for text in text_list:
    result = lm.do("What is the price in the following line, note sometimes it's not formatted and maybe werid, pls return empty if ther's no price seen" + text)
    price.append(result)


In [41]:
print(price)

price = [p for p in price if any(char.isdigit() for char in p)]
print(price)

['The price in the line is not provided.', "I'm sorry, but the price in the line is not provided. Please provide more information or context for me to assist you better.", "I'm sorry, but the price in the line is not provided. Please provide more information or context so that I can assist you better.", "I'm sorry, but the price in the line is not provided. Please provide more information or context so that I can assist you better.", "I'm sorry, but the price in the line is not provided. Please provide more information or context so that I can assist you better.", "I'm sorry, but the price in the line is not provided. Please provide more information or context for me to assist you better.", "I'm sorry, but the price in the line is not provided. Please provide more information or context so that I can assist you better."]
[]


In [44]:
import re
def highest_price(image):
    model = YOLO("runs/detect/train/weights/best.pt")
    bboxes = get_only_text_bboxes(model, image)
    bb = []
    for i in bboxes:
        bb.append(i.tolist()[0])
    sorted_bb = sorted(bb, key=lambda bbox: (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]))[::-1]
    img = cv2.imread('test/test.jpg')
    im_list= []
    for i in sorted_bb:
        im_list.append(crop_image(img, i))
    text_list = []
    for image in im_list:
        text = pytesseract.image_to_string(image)
        text_list.append(text)
    text_list = [p for p in text_list if any(char.isdigit() for char in p)]
    price = []
    for text in text_list:
        result = lm.do("What is the price in the following line, note sometimes it's not formatted and maybe werid, pls return empty if ther's no price seen" + text)
        price.append(result)
    price = [p for p in price if any(char.isdigit() for char in p)]
    digits = 0
    for p in price:
        match = re.search(r'\d+', p)
        if match:
            digits = match.group()
            break

    return digits
    
print("res: ", highest_price("test/test.jpg"))
    


image 1/1 /Users/kesh/Documents/GitHub/DarkPatterns/test/test.jpg: 320x640 3 Icons, 21 Images, 23 Texts, 41.3ms
Speed: 1.2ms preprocess, 41.3ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)
23378.992489445955
10412.595513090491
7906.868848504499
6432.538247443736
5593.983594536781
5172.420449277386
4804.493585953955
4145.43281686306
4012.3498050123453
3400.046914368868
2997.7621878739446
2931.799330741167
2458.4409774653614
2286.2599858324975
2083.2007139846683
2077.2622480094433
1933.8635388119146
1839.036332923919
1785.4994577616453
1577.305681295693
1377.3249604811426
1341.136989964638
1149.1021390631795
res:  0
